# Chisquare Test For Feature Selection
----------

The chi-square test is a statistical test used to determine whether there is a significant association between two categorical variables in a sample. It is also known as the chi-square goodness-of-fit test or chi-square independence test, depending on the specific application.

The test is based on the comparison between the observed frequencies (counts) of different categories in a contingency table and the expected frequencies that would be expected if the variables were independent. The contingency table is a two-dimensional table that cross-tabulates the categories of the two variables.

------------------

Compute chi-squared stats between each non-negative feature and class.

- This score should be used to evaluate categorical variables in a classification task.

This score can be used to select the n_features features with the highest values for the test chi-squared statistic from X, which must contain only non-negative features such as booleans or frequencies (e.g., term counts in document classification), relative to the classes.

Recall that the chi-square test measures dependence between stochastic variables, so using this function “weeds out” the features that are the most likely to be independent of class and therefore irrelevant for classification. The Chi Square statistic is commonly used for testing relationships between categorical variables.

It compares the observed distribution of the different classes of target Y among the different categories of the feature, against the expected distribution of the target classes, regardless of the feature categories.

In [67]:
import seaborn as sns
import numpy as np
import pandas as pd
import warnings as wr
df = sns.load_dataset('titanic')
wr.filterwarnings('ignore')

In [58]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,0,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,1,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,0,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,0,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,0,Third,man,True,NaN,Southampton,no,True


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [42]:
object_cols = ['sex','embarked','alone','pclass','survived']
df1 = df[object_cols]
df1.head(3)

,sex,embarked,alone,pclass,survived
0,male,S,False,3,0
1,female,C,False,1,1
2,female,S,True,3,1


### Label Encoding

Label encoding is a technique used in machine learning and data preprocessing to convert categorical variables into numerical representations. In this encoding scheme, each unique category is assigned a unique integer label.

In [43]:
from sklearn.preprocessing import LabelEncoder

In [44]:
label_encoder = LabelEncoder()

In [45]:
df1['sex']=label_encoder.fit_transform(df1['sex'])

In [46]:
df1.head(2)

,sex,embarked,alone,pclass,survived
0,1,S,False,3,0
1,0,C,False,1,1


In [47]:
df1.pclass.unique()

array([3, 1, 2], dtype=int64)

In [52]:
df1.survived.unique()

array([0, 1], dtype=int64)

In [54]:
df.embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [49]:
columns = ['embarked','alone','pclass','survived']
df1['embarked']=label_encoder.fit_transform(df1['embarked'])
df1['alone']=label_encoder.fit_transform(df1['alone'])
#df1['pclass']=label_encoder.fit_transform(df1['pclass'])
#df1['survived']=label_encoder.fit_transform(df1['survived'])
df1.head()

,sex,embarked,alone,pclass,survived
0,1,2,0,3,0
1,0,0,0,1,1
2,0,2,1,3,1
3,0,2,0,1,1
4,1,2,1,3,0


In [59]:
df1.isnull().sum()

sex         0
embarked    0
alone       0
pclass      0
survived    0
dtype: int64

In [61]:
X = df1.drop('survived', axis = 1 )
y = df1[['survived']]

from sklearn.model_selection import train_test_split as tts
X_train,X_test,y_train,y_test=tts(X, y,test_size=0.3, random_state = 0)

In [62]:
X_train.head()

,sex,embarked,alone,pclass
857,1,2,1,1
52,0,0,0,1
386,1,2,0,3
124,1,2,0,1
578,0,0,0,3


In [63]:
X_train.sex.unique()

array([1, 0])

In [64]:
X_train.isnull().sum()

sex         0
embarked    0
alone       0
pclass      0
dtype: int64

### Aplpy Chi^2 test


It returns 2 values:
- Fscore and
- pvalue

In [66]:
from sklearn.feature_selection import chi2
f_p_values=chi2(X_train, y_train)
f_p_values

(array([63.55447864,  6.2675888 ,  9.03328564, 21.61080949]),
 array([1.55992554e-15, 1.22966368e-02, 2.65107556e-03, 3.33964360e-06]))

In [74]:
p_values = pd.Series(f_p_values[1])
p_values.index = X_train.columns
p_values.sort_index(ascending=False)

sex         1.559926e-15
pclass      3.339644e-06
embarked    1.229664e-02
alone       2.651076e-03
dtype: float64

## Observation

Sex Column is the most important column when compared to the output feature Survived